<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#基本概念" data-toc-modified-id="基本概念-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>基本概念</a></span></li><li><span><a href="#实例" data-toc-modified-id="实例-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>实例</a></span></li><li><span><a href="#对文档进行分类" data-toc-modified-id="对文档进行分类-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>对文档进行分类</a></span><ul class="toc-item"><li><span><a href="#对文档分词" data-toc-modified-id="对文档分词-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>对文档分词</a></span></li><li><span><a href="#加载停用词" data-toc-modified-id="加载停用词-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>加载停用词</a></span></li><li><span><a href="#计算单词权重" data-toc-modified-id="计算单词权重-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>计算单词权重</a></span></li><li><span><a href="#生成分类器" data-toc-modified-id="生成分类器-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>生成分类器</a></span></li><li><span><a href="#分类器做预测" data-toc-modified-id="分类器做预测-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>分类器做预测</a></span></li><li><span><a href="#计算正确率" data-toc-modified-id="计算正确率-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>计算正确率</a></span></li></ul></li><li><span><a href="#练习" data-toc-modified-id="练习-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>练习</a></span></li></ul></div>

## 基本概念

高斯朴素贝叶斯（GauussianNB）：特征变量是**连续变量**，符合高斯分布，比如说人的身高、物体的长度。

多项式朴素贝叶斯（MultinomialNB）：特征变量是**离散变量**，符合多项分布，在文档分类中特征变量体现在一个单词出现的次数，或者是单词的 TF-IDF 值等。

伯努利朴素贝叶斯（BernoulliNB）：特征变量是**布尔变量**，符合 0/1 分布，在文档分类中是单词是否出现。

TF-IDF：Term Frequency 和 Inverse Document Frequency，分别代表词频和逆向文档频率。

TF：统计一个单词在文档中出现的次数，次数和单词重要性成正比。

IDF：指一个单词在文档中的区分度。它认为一个单词出现在的文档数越少，就越能通过这个单词把该文档和其他文档区分开。IDF 越大，代表该单词的区分度越大。

TF 和 IDF 的公式：![](https://static001.geekbang.org/resource/image/bc/4d/bc31ff1f31f9cd26144404221f705d4d.png)

（备注：IDF 的分母加 1，是因为有些单词不会出现在文档中，因此为了避免分母为 0，统一给单词出现的文档数加 1。）

## 实例

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer()

documents = [
    'this is the bayes documens',
    'this is the second second documents',
    'and the thrid one',
    'is this the document'
]

tfidf_matrix = tfidf_vec.fit_transform(documents)

print("不重复的词：", tfidf_vec.get_feature_names())
print("*" * 30)
print("每个单词的 ID：", tfidf_vec.vocabulary_)
print("*" * 30)
print("每个单词的 tfidf 值：", tfidf_matrix.toarray())

## 对文档进行分类

分类流程：
<img src="https://static001.geekbang.org/resource/image/25/c3/257e01f173e8bc78b37b71b2358ff7c3.jpg"  style="width: 1000px;"/>

### 对文档分词

英文文档，使用 NTLK 包：

In [ ]:
import nltk

word_list = nltk.word_tokenize(text)
nltk.pos_tag(word_list)

中文文档，使用 jieba 包：

In [ ]:
import jieba

word_list = jieba.cut(text)

### 加载停用词

In [ ]:
stop_words = [line.strip().decode('utf-8') for line in io.open('stop_word.txt').readlines()]

### 计算单词权重

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(stop_words=stop_words, max_df=0.5)
'''
max_df：描述单词在文档中的最高出现率。
max_df=0.5，代表一个单词在 50% 的文档中出现过，那么它只携带了非常少的信息，因此不作为分词统计。
一般很少设置 min_df，因为 min_df 通常很小。
'''

### 生成分类器

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=0.001).fit(train_features, train_labels)

当 alpha=1 时，使用 Laplace 平滑，即采用加 1 的方式，来统计没有出现过的单词的概率。这样当训练样本很大的时候，加 1 得到的概率变化可以忽略不计，也同时避免了零概率的问题。

当 0<alpha<1 时，使用 Lidstone 平滑。对于 Lidstone 平滑来说，alpha 越小，迭代次数越多，精度越高。我们可以设置 alpha 为 0.001。

### 分类器做预测

创建一个 TfidfVectorizer 类，使用相同的 stop_words 和 max_df，而后用这个 TfidfVectorizer 类对测试集的内容进行 fit_transform 拟合，得到测试集的特征矩阵 test_features。

然后，用训练好的分类器对新数据做预测。使用 predict 函数求解所有后验概率并找出最大的那个。

In [ ]:
test_tf = TfidfVectorizer(stop_words=stop_words,
                          max_df=0.5, vocabulary=train_vocabulary)
test_features = test_tf.fit_transform(test_contents)


predicted_labels = clf.predict(test_features)

### 计算正确率

使用 sklearn 的 metrice 包中的 accuracy_score 函数，对实际结果和预测结果做对比，给出模型的准确率。

In [ ]:
from sklearn import metrics

print(metrics.accuracy_score(test_labels, predicted_labels))

## 练习

1. 使用朴素贝叶斯对提供的训练集进行训练，并对测试集进行验证、给出测试集的准确率。代码如下：

1. 判断一个人的性别时，适合使用哪种分类器？多项式朴素贝叶斯。

1. 停用词的作用又是什么？节省空间和计算时间。